# Explore first version of MOTBX content

In [ ]:
from pathlib import Path
import os
import pprint
pp = pprint.PrettyPrinter(indent=2, width=80, compact=True)


CWD = Path.cwd()
if CWD.name != "notebooks":
    print("Make sure to run this notebook from the 'notebooks' directory.")
MOTBX_DIR = CWD.parent
MOTBX_LEGACY_XLSX = MOTBX_DIR.joinpath(
    "resources/legacy/MOTBX_resources_for_website.xlsx")
RESOURCES_DIR = MOTBX_DIR.joinpath("resources/curated")
if not os.path.exists(RESOURCES_DIR):
    os.mkdir(RESOURCES_DIR)
SCHEMA_JSON = MOTBX_DIR.joinpath("schema/motbxschema.json")


Load sheet from Excel file and print information about colummns and field values.

In [ ]:
import pandas as pd

content_df = pd.read_excel(
    MOTBX_LEGACY_XLSX, sheet_name="website_content_for_JB", header=0)
content_df.head()

In [ ]:
# column names
list(content_df.columns)

In [ ]:
# category names
sorted(set(content_df["Category"]))

In [ ]:
# subcategory names
pp.pprint(sorted([len(i) for i in content_df["Sub-category"]]))
sorted(set(content_df["Sub-category"]))

In [ ]:
# category and subcategory names
set([(i["Category"], i["Sub-category"]) for idx, i in content_df.iterrows()])

In [ ]:
# lengths of titles
pp.pprint(sorted([len(i) for i in content_df["Resource_title"]]))

In [ ]:
# lengths of descriptions
pp.pprint(sorted([len(i) for i in content_df["Resource_description"]]))

In [ ]:
# tags
pp.pprint(sorted(set([j.strip() for i in content_df["Tags"] 
    for j in i.split(",")])))

In [ ]:
# category names
sorted(set([i for i in content_df["Doc_type"] if isinstance(i, str)]))

In [ ]:
# category names
sorted(set([i for i in content_df["Format"] if isinstance(i, str)]))

Create YAML files from data frame rows.

In [ ]:
from motbxtools import motbxschema
import jsonschema
import yaml

schema = motbxschema.MotbxSchema(SCHEMA_JSON)

In [ ]:
resources_to_check = []
for idx, row in content_df.iterrows():
    row = row.str.strip()
    resource_id = f"ID{idx+1:04n}"
    resource = {
        "resourceID": resource_id,
        "resourceCategory": row["Category"],
        "resourceSubcategory": row["Sub-category"],
        "resourceTitle": row["Resource_title"],
        "resourceDescription": row["Resource_description"],
        "resourceUrl": row["Link_to_resource"].replace("http://", "https://"),
        "resourceTags": sorted([i.strip() for i in row["Tags"].split(",")]), 
    }
    if resource["resourceUrl"] != row["Link_to_resource"]:
        resources_to_check.append(
            (resource_id, row["Resource_title"], 
            f"NOTE: Use https instead of http in {row['Link_to_resource']}"))
    # validate against JSON schema
    try:
        jsonschema.validate(resource, schema.schema)
    except Exception as e:
        # print("--------------------\n", resource_id, row["Resource_title"])
        # print(e)
        resources_to_check.append((resource_id, row["Resource_title"], str(e)))
        continue
    # save to file
    yaml_path = RESOURCES_DIR.joinpath(f"{resource_id}.yaml")
    with open(yaml_path, "w") as fp:
        yaml.dump(resource, fp)

In [ ]:
import csv

# write warnings and errors about resources to file
out_file = MOTBX_DIR.joinpath(
    "resources/legacy/NOTES_on_MOTBX_resources_for_website.csv")
with open(out_file, "w") as fp:
    writer = csv.writer(fp, delimiter='\t', quotechar='"')
    for row in resources_to_check:
        writer.writerow(["-"*80])
        writer.writerow(row)

In [ ]:

with open(MOTBX_DIR.joinpath("categories_tags.yaml"), "r") as fp:
    tags = yaml.safe_load(fp)

#print(yaml.dump(tags))
pp.pprint(tags)




In [ ]:
import itertools

def browse_tags(tags, parents = []):
    if isinstance(tags, dict):
        for k, v in tags.items():
            if k != "synonyms":
                yield from browse_tags(v, parents = parents + [k])
            else:
                yield f'{"_".join(parents[:-1])}|{parents[-1]}|{"_".join(v)}'
    elif isinstance(tags, list):
        l = [browse_tags(i, parents = parents) for i in tags]
        yield from itertools.product(*l)
    elif isinstance(tags, str):
        yield f'{"_".join(parents)}|{tags}|'
    else:
        print("else", tags)
        
    
def flatten(i):
    if isinstance(i, tuple):
        for j in i:
            yield from flatten(j)
    else:
        yield i

for i in browse_tags(tags):
    for j in flatten(i):
        print(j)


In [ ]:
tags = {
    'translational research data': [
        {'omics': [
            {'synonyms': ['type of omics', 'omics type', 'experimental data']},
            {'genomics': [
                    {'synonyms': ['genome']}
                ]}
            
        ]}
    ]
}


print(yaml.dump(tags))






